# Chapter 19: Analyzing Financial Data Across Asset Classes

This chapter embarks on a captivating journey, equipping you with the skills to analyze financial data across a diverse range of asset classes. We'll delve into the power of the Tiingo API, a gateway to unlocking historical price data for stocks, ETFs, cryptocurrencies, and foreign exchange (forex).

Our arsenal will include the pandas library for data manipulation, the versatile matplotlib library for creating visualizations, and cufflinks (set to offline mode) for interactive plotting. By the chapter's conclusion, you'll be adept at:

Retrieving Historical Data: Effortlessly querying historical price data for various assets.

Price Normalization: Standardizing prices to facilitate insightful comparisons across assets.

Correlation Visualization: Unveiling hidden relationships between different financial instruments using heatmaps and scatter plots.

## 19.1 Imports

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tiingo import TiingoClient
import cufflinks as cf                   # Interactive plotting library (set to offline mode here).
cf.set_config_file(offline=True)         # Disables cufflinks' online functionalities for use with pre-downloaded data.

Cufflinks is a Python library that seamlessly integrates with Pandas and Plotly to create interactive visualizations directly from DataFrames.

It simplifies the process of transforming data into compelling visual representations.   

Key Features:

**Integration with Pandas:** Cufflinks operates directly on Pandas DataFrames, allowing for effortless plotting.

**Interactive Plots:** It generates interactive visualizations that can be explored and analyzed in detail.

**Plotly Backend:** Leveraging Plotly's capabilities, Cufflinks provides a wide range of plot types and customization options.

**Offline Functionality:** You can configure Cufflinks to work in offline mode, making it suitable for environments without an active internet connection.   

In essence, Cufflinks acts as a user-friendly interface between your data and visually appealing, interactive charts.

In [ ]:
# API Connection
os.environ['TIINGO_API_KEY'] = 'YOUR KEY'
client = TiingoClient()

## 19.2 Stocks

In [ ]:
# Stock Price Data
symbol = 'AAPL'
prices = client.get_dataframe(symbol, startDate='2021-01-01', endDate='2023-12-31', frequency='daily')
price_df = prices[['adjClose']].rename(columns={'adjClose': 'adjusted_close'})
price_df.info()
price_df.tail()                          #Shows the last 1000 closing prices using price_df.tail().
price_df['adjusted_close'].iloc[-1000:].iplot(title=symbol)

## 19.3 ETFs

Reminder: Exchange-Traded Funds (ETFs) are investment funds that trade on stock exchanges, much like individual stocks, and typically aim to track the performance of a specific index or sector. They offer investors diversification, liquidity, and often lower fees compared to mutual funds.

In [ ]:
# This module contains the basic building blocks for creating plots in plotly, such as Figure, Scatter, Bar, and other plot types. 
import plotly.graph_objs as go 

# This imports the iplot function from the plotly.offline module. The iplot function is used to display plots inline within a Jupyter notebook. 
from plotly.offline import iplot

# ETF Data
etf_symbols = [
    'IYW',                  # iShares U.S. Technology ETF
    'QQQ',                  # Invesco QQQ Trust
    'SPY',                  # SPDR S&P 500 ETF Trust
    'GLD',                  # SPDR Gold Shares
    'VTI',                  # Vanguard Total Stock Market ETF
    'VEA',                  # Vanguard FTSE Developed Markets ETF
    'BND'                   # Vanguard Total Bond Market ETF
]

etf_data = {}
for etf in etf_symbols:
    prices = client.get_dataframe(etf, startDate='2021-01-01', endDate='2023-12-31', frequency='daily')
    price_df = prices[['adjClose']].rename(columns={'adjClose': 'close'})
    etf_data[etf] = price_df

etf_df = pd.DataFrame()
for etf in etf_data:
    etf_df[etf] = etf_data[etf]['close']
etf_df.dropna(inplace=True)

# Normalize the ETF prices
etf_df_normalized = etf_df / etf_df.iloc[0]

# Plot normalized ETF prices
fig = go.Figure()
for etf in etf_df_normalized.columns:
    fig.add_trace(go.Scatter(x=etf_df_normalized.index, y=etf_df_normalized[etf], mode='lines', name=etf))

fig.update_layout(title='Normalized ETF Prices',
                  xaxis_title='Date',
                  yaxis_title='Normalized Price')
iplot(fig)

# Print correlation matrix
print(etf_df_normalized.corr())

# Create a heatmap visualizing the correlation matrix of the normalized ETF prices
etf_corr = etf_df_normalized.corr()
heatmap = go.Figure(data=go.Heatmap(
                   z=etf_corr.values,
                   x=etf_corr.index.values,
                   y=etf_corr.columns.values,
                   colorscale='Blues'))
heatmap.update_layout(title='Correlation of ETF Prices')
iplot(heatmap)


## 19.4 Cryptocurrency

Reminder: Cryptocurrency is a digital or virtual form of currency that uses cryptography for secure transactions and operates independently of a central authority, like a government or bank. It relies on blockchain technology to maintain a decentralized and transparent ledger of transactions.

In [ ]:
# Cryptocurrency Data
crypto_symbols = [
    'ethusd',                                      # Ethereum
    'xrpusd',                                      # Ripple
    'ltcusd',                                      # Litecoin
    'bchusd',                                      # Bitcoin Cash
    'adausd'                                       # Cardano
]

crypto_data = {}
for crypto in crypto_symbols:
    prices = client.get_dataframe(crypto, startDate='2021-01-01', endDate='2023-12-31', frequency='daily')
    price_df = prices[['adjClose']].rename(columns={'adjClose': 'close'})
    crypto_data[crypto] = price_df

crypto_df = pd.DataFrame()
for crypto in crypto_data:
    crypto_df[crypto] = crypto_data[crypto]['close']

crypto_df.dropna(inplace=True)

# Normalize the prices
crypto_df_normalized = crypto_df / crypto_df.iloc[0]

# Plot normalized prices
fig = go.Figure()
for crypto in crypto_df_normalized.columns:
    fig.add_trace(go.Scatter(x=crypto_df_normalized.index, y=crypto_df_normalized[crypto], mode='lines', name=crypto))

fig.update_layout(title='Normalized Cryptocurrency Prices',
                  xaxis_title='Date',
                  yaxis_title='Normalized Price')
iplot(fig)

# Print correlation matrix
print(crypto_df_normalized.corr())

# Plot the Cryptocurrency normalized prices correlation
crypto_corr = crypto_df_normalized.corr()
crypto_heatmap = go.Figure(data=go.Heatmap(
                   z=crypto_corr.values,
                   x=crypto_corr.index.values,
                   y=crypto_corr.columns.values,
                   colorscale='Blues'))
crypto_heatmap.update_layout(title='Correlation of Cryptocurrency Prices')
iplot(crypto_heatmap)

## 19.5 Forex

Reminder: Foreign exchange, or forex, is the global marketplace for trading national currencies against one another, driven by fluctuations in exchange rates. It operates 24/5, allowing traders to buy, sell, and exchange currencies with the aim of capitalizing on changes in their relative values.

In [ ]:
# Forex Data
forex_symbols = [
    'EURUSD',                                               # Euro to USD
    'GBPUSD',                                               # British Pound to USD
    'USDCAD',                                               # USD to Canadian Dollar
    'USDCHF',                                               # USD to Swiss Franc
    'AUDUSD'                                                # Australian Dollar to USD
]

forex_data = {}
for forex in forex_symbols:
    prices = client.get_dataframe(forex, startDate='2021-01-01', endDate='2023-12-31', frequency='daily')
    price_df = prices[['adjClose']].rename(columns={'adjClose': 'close'})
    forex_data[forex] = price_df

forex_df = pd.DataFrame()
for forex in forex_data:
    forex_df[forex] = forex_data[forex]['close']
forex_df.dropna(inplace=True)

# Normalize the forex prices
forex_df_normalized = forex_df / forex_df.iloc[0]

# Plot normalized Forex prices
fig = go.Figure()
for forex in forex_df_normalized.columns:
    fig.add_trace(go.Scatter(x=forex_df_normalized.index, y=forex_df_normalized[forex], mode='lines', name=forex))

fig.update_layout(title='Normalized Forex Prices',
                  xaxis_title='Date',
                  yaxis_title='Normalized Price')
iplot(fig)

# Print correlation matrix
print(forex_df_normalized.corr())

# Create a heatmap visualizing the correlation matrix of the normalized Forex prices
forex_corr = forex_df_normalized.corr()
heatmap = go.Figure(data=go.Heatmap(
                   z=forex_corr.values,
                   x=forex_corr.index.values,
                   y=forex_corr.columns.values,
                   colorscale='Blues'))
heatmap.update_layout(title='Correlation of Forex Prices')
iplot(heatmap)

## 19.6 Summary

This chapter provides an in-depth guide on how to analyze and compare financial data across various asset classes, such as stocks, ETFs, cryptocurrencies, and foreign exchange (forex). Utilizing the **Tiingo API**, **pandas** for data manipulation, **matplotlib** for visualizations, and **cufflinks** for interactive plotting, we gain insights into the relationships between these assets and explore their historical price data.